This script generate data of customer in may 2015 who buy additional product since april 2015

In [1]:
import numpy as np
import pandas as pd 
from collections import defaultdict

In [2]:
train_cols = ['fecha_dato',
 'ncodpers','ind_empleado','pais_residencia','sexo','age','fecha_alta','ind_nuevo','antiguedad','indrel',
 'indrel_1mes','tiprel_1mes','indresi','indext','conyuemp','canal_entrada','indfall',
 'tipodom','cod_prov','ind_actividad_cliente','renta','segmento',
 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']

notuse = ["ult_fec_cli_1t","nomprov"]

product_col = [
 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']


In [3]:
df_train = pd.read_csv('input/train_ver2.csv',usecols=train_cols)

/Users/rkcosmos/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
df_train_may = df_train[df_train['fecha_dato']=='2015-05-28']
df_train_june = df_train[df_train['fecha_dato']=='2015-06-28']

df_train.drop('fecha_dato', axis=1, inplace=True)

In [6]:
dfm = pd.merge(df_train_june,df_train_may, how='left', on=['ncodpers'], suffixes=('', '_prev'))

In [7]:
prevcols = [col for col in dfm.columns if '_ult1_prev' in col]
currcols = [col for col in dfm.columns if '_ult1' in col and '_ult1_prev' not in col]

In [8]:
for col in prevcols:
    dfm[col].fillna(0, inplace=True)

In [9]:
for col in currcols:
    dfm[col].fillna(0, inplace=True)

In [10]:
for col in currcols:
    dfm[col] = dfm[col] - dfm[col+'_prev']
    dfm[col] = dfm[col].apply(lambda x: max(x,0))

In [11]:
prevcols2 = [col for col in dfm.columns if '_prev' in col and col not in prevcols]
dfm.drop(prevcols2, axis=1, inplace=True)


In [12]:
dfm = dfm[dfm[currcols].sum(axis=1) >0]

In [13]:
dfm[currcols].sum().sum()

45679.0

In [14]:
df_new = pd.DataFrame()

for index, row in dfm.iterrows():
    if index%1000 ==0:
        print(index)
    for i,col in enumerate(currcols):
        if row[col] == 1:
            row['target'] = currcols.index(col)
            df_new = df_new.append(row)
            
df_new.drop(currcols, axis=1, inplace=True)

7000
40000
48000
55000
63000
104000
106000
192000
210000
211000
212000
215000
222000
235000
253000
273000
280000
293000
319000
320000
339000
340000
371000
385000
396000
397000
417000
447000
456000
504000
526000
528000
550000
553000
558000
564000
566000
570000
581000
597000
612000
627000


In [15]:
df_new.to_csv('cleaned_data/juneExtraMulticlass.csv')

In [16]:
df_new.shape

(45679, 47)